In [1]:
import numpy as np
from pathlib import Path
from functools import reduce
from numpy.lib.stride_tricks import sliding_window_view

In [2]:
def load_puzzle_input(path):
    puzzle_input_path = Path(path)
    warehouse_map = []
    moves = []
    moves_mapping = { '>': np.array([0, 1]), '^': np.array([-1, 0]), '<': np.array([0, -1]), 'v': np.array([1, 0])}

    with open(puzzle_input_path, 'r') as f:
        is_move_line = False
        for row, line in enumerate(f):
            line = line.strip()
            if row == 0:
                continue
            if line == ('#' * len(line)):
                is_move_line = True
                continue
            if not is_move_line:
                warehouse_map.append(list(line[1:-1]))
            else:
                moves.extend([moves_mapping[move] for move in line])
    return np.array(warehouse_map), moves

In [149]:
class WharehouseMap:
    def __init__(self, warehouse_map):
        self.warehouse_map = warehouse_map.copy()
        self.shape_array = np.array(self.warehouse_map.shape)

    def __str__(self):
        return '\n'.join(''.join(map(str, row)) for row in self.warehouse_map)

    def _move_robot(self, idx_position, idx_new_position):
        self.warehouse_map[idx_position], self.warehouse_map[idx_new_position] = '.', self.warehouse_map[idx_position]
        return True


    def move(self, position: np.ndarray, move: np.ndarray):
        new_position = position + move
        if ((new_position < 0) | (new_position >= self.shape_array)).any():
            return False
        idx_position, idx_new_position = tuple(position), tuple(new_position)

        match self.warehouse_map[idx_new_position]:
            case '.':
                return self._move_robot(idx_position, idx_new_position)
            case '#':
                return False
            case 'O':
                return self._move_robot(idx_position, idx_new_position) if self.move(new_position, move) else False
            case '[':
                end_box = new_position + np.array([0, 1])
                moves_map = self._can_push_box(new_position, end_box, move)
                return self._move_robot(idx_position, idx_new_position) if self._push_box(moves_map) else False
            case ']':
                start_box = new_position + np.array([0, -1])
                moves_map = self._can_push_box(start_box, new_position, move)
                return self._move_robot(idx_position, idx_new_position) if self._push_box(moves_map) else False

    def _can_push_box_x(self, start_box, end_box, move):
        start_box_new, end_box_new = start_box + move, end_box + move
        start, end = tuple(start_box), tuple(end_box)
        start_new, end_new = tuple(start_box_new), tuple(end_box_new)
        (point, box_side) = (end_new, '[') if move[1] > 0 else (start_new, ']')

        if self.warehouse_map[point] == '.':
            return [WharehouseMap.step(start, end, start_new, end_new)]

        if self.warehouse_map[point] == box_side:
            steps = self._can_push_box(start_box_new + move, end_box_new + move, move)
            if not steps:
                return []
            steps.append(WharehouseMap.step(start, end, start_new, end_new))
            return steps

        return []

    def _steps_to_push_box(self, start_box, end_box, move=np.array([0, 0]), offset = np.array([0, 0])):
        start_box_new, end_box_new = start_box + move, end_box + move
        steps = self._can_push_box(start_box_new + offset, end_box_new + offset, move)
        if not steps:
            return []
        steps.append(WharehouseMap.step(start_box, end_box, start_box_new, end_box_new))
        return steps

    @staticmethod
    def step(start, end, start_new, end_new):
        return ({
            'from': ([start[0], end[0]] , [start[1], end[1]]),
            'to': ([start_new[0], end_new[0]] , [start_new[1], end_new[1]])
        })

    def _can_push_box(self, start_box, end_box, move):
        start_box_new, end_box_new = start_box + move, end_box + move
        for pos in (start_box_new, end_box_new):
            if (pos < 0).any() or (pos >= self.shape_array).any():
                return []

        start, end = tuple(start_box), tuple(end_box)
        start_new, end_new = tuple(start_box_new), tuple(end_box_new)

        match tuple(move):
            case (0, _):
                return self._can_push_box_x(start_box, end_box, move)

            case _:
                match (self.warehouse_map[start_new], self.warehouse_map[end_new]):
                    case ('#', _) | (_, '#'):
                        return []

                    case ('.', '.'):
                        return [WharehouseMap.step(start, end, start_new, end_new)]

                    case ('.', '['):
                        return self._steps_to_push_box(start_box, end_box, move, np.array([0, 1]))

                    case ('[', ']'):
                        return self._steps_to_push_box(start_box, end_box, move)

                    case (']', '.'):
                        return self._steps_to_push_box(start_box, end_box, move, np.array([0, -1]))

                    case (']', '['):
                        steps = []
                        for column_offset in [np.array([0, 1]), np.array([0, -1])]:
                            _steps = self._can_push_box(start_box_new + column_offset, end_box_new + column_offset, move)
                            if not _steps:
                                return []
                            steps.extend(_steps)
                        steps.append(WharehouseMap.step(start, end, start_new, end_new))
                        return steps
                    case _:
                        return []

    def _push_box(self, moves):
        if not moves:
            return False
        for move in moves:
            # if '[' not in self.warehouse_map[move['from']] or ']' not in self.warehouse_map[move['from']]:
            if (self.warehouse_map[move['from']] != np.array(['[',']'])).any():
                continue
            self.warehouse_map[move['to']] = self.warehouse_map[move['from']]
            if move['to'][0] != move['from'][0]:
                self.warehouse_map[move['from']] = '.'
            else:
                columns = [column for column in move['from'][1] if not column in move['to'][1]]
                self.warehouse_map[move['to'][0][0], columns] = '.'
        return True

In [150]:
warehouse_map, moves = load_puzzle_input('inputs/day15.txt')
robot = np.where(warehouse_map == '@')
robot_position = np.array(list(zip(robot[0], robot[1]))[0])
move_map = WharehouseMap(warehouse_map)
for move in moves:
    if move_map.move(robot_position, move):
        robot_position += move

In [5]:
indices = np.where(move_map.warehouse_map == 'O')
reduce(lambda acc, value: acc + 100 * (value[0] + 1) + (value[1] + 1), zip(indices[0], indices[1]), 0)

1448589

In [156]:
warehouse_map, moves = load_puzzle_input('inputs/day15.txt')
warehouse_map_extend = np.repeat(warehouse_map, repeats=2, axis=1)
cols = np.arange(warehouse_map_extend.shape[1])
even_cols = cols % 2 == 0
warehouse_map_extend[(warehouse_map_extend == 'O') & (even_cols[np.newaxis])] = '['
warehouse_map_extend[(warehouse_map_extend == 'O') & (~even_cols[np.newaxis])] = ']'
# windows = sliding_window_view(warehouse_map_extend, (1, 2))
# idx = np.argwhere(np.all(windows == np.array(['@', '@']), axis=(2, 3)))
duplicate_robot_idx = tuple((np.argwhere(warehouse_map == '@') * np.array([[1, 2]]))[0] + np.array([0, 1]))
warehouse_map_extend[duplicate_robot_idx] = '.'
move_map_extend = WharehouseMap(warehouse_map_extend)
robot = np.where(warehouse_map_extend == '@')
robot_position = np.array(list(zip(robot[0], robot[1]))[0])

# print(move_map_extend, '==============================', sep='\n')

for move in moves:
    if move_map_extend.move(robot_position, move):
        robot_position += move

    # print(move)
    # print(move, move_map_extend, '==============================', sep='\n')

print(move)
indices = np.where(move_map_extend.warehouse_map == '[')
reduce(lambda acc, value: acc + 100 * (value[0] + 1) + (value[1] + 2), zip(indices[0], indices[1]), 0)

[0 1]


1472235

In [157]:
print(move_map_extend)

[]####[][][]..[]..........[][][][][][]##[][][][][]##..[]..........[]....[]..........[]........[]
[][][]....[]................[][]##[]..[].[].[][]..[]##[][]##..[]####......................[][]..
[]........[]####[]...........[][][][]...[].[]..[][][]...##....[]..[]..[]..##..........##..[]##..
....##..[]..[][]##............[]...........[][][].......[][][]##..[]................[]..[]##[]..
....##..##......[]##......................[]......................[]..[]........................
................[][]...[]...[]##.......................[][].[]..[]......[]....[]........[][][][]
..##............[]....##....##....................##..[][]........##....##..[][]....[]..[]......
[][][]..[]............................##..##.......[].........[]............[][]....[]##[]..##[]
..[]..####[][]......................[]##...........[].........[]..[]....[][]........[]##......##
##[].................................[]............[].....####[]..[]....[]..........##..[][]....
.............[]..[].......[]..

In [ ]:
move_map_extend.warehouse_map

False